# 01a ReadData_Discussion

Contents
 - Import packages
 - Review of memory requirements and issues
 
Notes
 - 


**Import packages and set directories**

In [2]:
# system commands
import os

# load and save data
import pickle

# data manipulation
import numpy as np
import pandas as pd

# graphs
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
print(np.__version__)
print(pd.__version__)

1.19.5
1.2.4


In [4]:
# Set directories
print(os.getcwd())
dirRawData = "../input/"
dirPData   = "../PData/"

/home/jovyan/Projects/Fraud/PCode


**Subsets of data**

Even if we can store the whole data (see the details below), copying it to h2o and then running various models seems to take something like 16 gig. Also full runs can take a long time, so it is better to run initial experiments on a subset of the data. 

Therefore subsets with 250k records (train_250k) and 1m records (train_1m) were previously created (01a ReadData_Downsample_Test).

**Memory issues**

When running Docker on mac, the default maximum memory is set to 2gig.  This can be seen (and changed) in the Docker Dashboard.  The size of the Pandas dataframe required to stored the data when using default pd.read_csv is around 1.5 gig, BUT the memory usage whilst reading this in, peaks at close to 4 gig.  This can be seen by running the command "docker stats" from the terminal command line and watching as the data is read in.

Without enough memory available to your container you will just get an error - something like "kernel died and needs to restart", or something like that.  The error will not make it clear that you ran out of memory.  

So I began to look at memory usage of the data frame.  I should mention that at this point, I had missings in many of the numerics.

Having read in the data, I then looked at the dtypes.  Strings were stored as objects.  Numbers were stored as float64 or int64.  Both of these are much larger than needed.  I therefore printed the dict of dtypes, using "df_train.dtypes.to_dict()" and manually changed the dtypes as necessary.

Changing just dtype Object to category reduced the memory of df_train to 1.5 gig to 0.831 gig.  However, this doing this is problematic since, the creation of categorical separately for test data could create different categoricals.

Most of the integers have a range -127 to 127 and so can be stored with int8.  a04 goes up to 240 and so can be stored with a uint8 or int16.  Unfortunately (as discussed at the start of Week 4), NA's will cause int's to be upcast to floats - since NA is a float.  Downcasting those we can (because there are no missings) to int8, takes  us down to 0.5 gig. At this stage I went back to the original data and replaced all missing numbers with -99 (having checked that this is not otherwise in those fields).

After this - size is 0.203 gig.  (Using object type instead of category, size is 0.77 gig)

Datasets here are smaller since limited to 250k or 1m

In [5]:
dtypes_cat = {
 'target': 'int8',
 'a01': "category",
 'a02': "category",
 'a03': "category",
 'a10': "category",
 'a11': "int8",
 'a12': "category",
 'a13': "category",
 'a16': "category",
 'a17': "category",
 'a18': "category",
 'a19': "category",
 'a20': "category",
 'b02': "category",
 'b03': "category",
 'b04': "category",
 'b07': "category",
 'c02': "category",
 'c04': "category",
 'c05': "category",
 'c06': "category",
 'c07': "category",
 'c08': "category",
 'c09': "category",
 'e01': "category",
 'e03': "category",
 'e11': "category",
 'e13': "category",
 'e14': "category",
 'e21': "category",
 'e22': "category",
 'e24': "category",
 'e25': "category",
 'e17': "category",
 'e18': "category",
 'e19': "category",
 'e20': "category",
 'f03': "category",
 'f04': "category",
 'f05': "category",
 'f07': "category",
 'f09': "category",
 'f27': "category",
 'f29': "category",
 'f30': "category",
 'f33': "category",
 'f34': "category",
 'f10': "category",
 'a04': 'int16',
 'a05': 'int8',
 'a06': 'int8',
 'a07': 'int8',
 'a08': 'int8',
 'a09': 'int8',
 'a14': 'int8',
 'a15': 'int8',
 'b01': 'int8',
 'b05': 'int8',
 'b06': 'int8',
 'c01': 'int8',
 'c03': 'int8',
 'd01': 'int8',
 'd02': 'int8',
 'd03': 'int8',
 'e02': 'int8',
 'e04': 'int8',
 'e05': 'int8',
 'e06': 'int8',
 'e07': 'int8',
 'e08': 'int8',
 'e09': 'int8',
 'e12': 'int8',
 'e15': 'int8',
 'e16': 'int8',
 'e23': 'int8',
 'f01': 'int8',
 'f02': 'int8',
 'f06': 'int8',
 'f08': 'int8',
 'f11': 'int8',
 'f13': 'int8',
 'f15': 'int8',
 'f16': 'int8',
 'f17': 'int8',
 'f18': 'int8',
 'f19': 'int8',
 'f20': 'int8',
 'f21': 'int8',
 'f22': 'int8',
 'f23': 'int8',
 'f24': 'int8',
 'f25': 'int8',
 'f26': 'int8',
 'f28': 'int8',
 'f31': 'int8',
 'f32': 'int8',
 'unique_id': 'int64'}


In [6]:
dtypes_object = {
 'a01': "O",
 'a02': "O",
 'a03': "O",
 'a10': "O",
 'a11': "int8",
 'a12': "O",
 'a13': "O",
 'a16': "O",
 'a17': "O",
 'a18': "O",
 'a19': "O",
 'a20': "O",
 'b02': "O",
 'b03': "O",
 'b04': "O",
 'b07': "O",
 'c02': "O",
 'c04': "O",
 'c05': "O",
 'c06': "O",
 'c07': "O",
 'c08': "O",
 'c09': "O",
 'e01': "O",
 'e03': "O",
 'e11': "O",
 'e13': "O",
 'e14': "O",
 'e21': "O",
 'e22': "O",
 'e24': "O",
 'e25': "O",
 'e17': "O",
 'e18': "O",
 'e19': "O",
 'e20': "O",
 'f03': "O",
 'f04': "O",
 'f05': "O",
 'f07': "O",
 'f09': "O",
 'f27': "O",
 'f29': "O",
 'f30': "O",
 'f33': "O",
 'f34': "O",
 'f10': "O",
 'a04': 'int16',
 'a05': 'int8',
 'a06': 'int8',
 'a07': 'int8',
 'a08': 'int8',
 'a09': 'int8',
 'a14': 'int8',
 'a15': 'int8',
 'b01': 'int8',
 'b05': 'int8',
 'b06': 'int8',
 'c01': 'int8',
 'c03': 'int8',
 'd01': 'int8',
 'd02': 'int8',
 'd03': 'int8',
 'e02': 'int8',
 'e04': 'int8',
 'e05': 'int8',
 'e06': 'int8',
 'e07': 'int8',
 'e08': 'int8',
 'e09': 'int8',
 'e12': 'int8',
 'e15': 'int8',
 'e16': 'int8',
 'e23': 'int8',
 'f01': 'int8',
 'f02': 'int8',
 'f06': 'int8',
 'f08': 'int8',
 'f11': 'int8',
 'f13': 'int8',
 'f15': 'int8',
 'f16': 'int8',
 'f17': 'int8',
 'f18': 'int8',
 'f19': 'int8',
 'f20': 'int8',
 'f21': 'int8',
 'f22': 'int8',
 'f23': 'int8',
 'f24': 'int8',
 'f25': 'int8',
 'f26': 'int8',
 'f28': 'int8',
 'f31': 'int8',
 'f32': 'int8',
 'unique_id': 'int64'}

In [7]:
df_train_1m = pd.read_csv(dirRawData + 'train_1m.csv',
                          na_values = 'NA',
                          dtype=dtypes_cat)


In [8]:
df_train_1m.shape

(1000000, 97)

In [9]:
df_train_1m.head(3)

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f21,f22,f23,f24,f25,f26,f28,f31,f32,unique_id
0,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,0,0,0,5,0,0,1
1,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,0,0,0,5,0,0,2
2,0,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,0,0,0,5,0,0,3


In [10]:
# How much memory is df_train_1m using?
mem_used = df_train_1m.memory_usage().sum() / (2<<29)
print("df_train_1m is using {:.3f} gig".format(mem_used))

del mem_used

df_train_1m is using 0.104 gig


In [11]:
print('df_train_1m: ', df_train_1m.shape)

df_train_1m:  (1000000, 97)


In [12]:
df_train_1m.describe()

,target,a04,a05,a06,a07,a08,a09,a11,a14,a15,...,f21,f22,f23,f24,f25,f26,f28,f31,f32,unique_id
count,1000000.000000,1000000.000000,1000000.000000,1000000.00000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06
mean,0.491048,7.066254,-9.367583,-9.32703,-9.385986,-0.728543,-0.759805,-9.439043,-8.882882,-1.211314,...,0.007523,0.054844,0.002139,0.006346,0.006593,0.006827,4.866512,-0.053210,-0.045587,5.752154e+05
std,0.499920,105.383285,29.035928,29.05040,29.029318,8.555270,9.115909,29.097942,29.582418,21.832745,...,0.088082,0.263397,0.046609,0.079798,0.081887,0.087741,0.736763,0.476026,0.460255,3.322109e+05
min,0.000000,-99.000000,-99.000000,-99.00000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,-3.000000,1.000000e+00
25%,0.000000,-99.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,2.873428e+05
50%,0.000000,12.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,5.750435e+05
75%,1.000000,60.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,8.628095e+05
max,1.000000,240.000000,3.000000,3.00000,1.000000,2.000000,2.000000,2.000000,3.000000,8.000000,...,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,5.000000,3.000000,3.000000,1.150802e+06


**Checking the cardinality**

For my own work, I decided to start of with anything above cardinality of 30 to be treated as an hccv. This is not necessarily optimal and you should feel to ignore this.

In [13]:
cols_numeric = list(df_train_1m.select_dtypes(include=[np.number]).columns.values)
cols_notNumeric = [var for var in df_train_1m.columns if var not in cols_numeric]
srs_card = df_train_1m[cols_notNumeric].nunique()
print(srs_card.min())
print(srs_card.max())
srs_card[srs_card>30]

2
35575


e17       95
e18      820
e19    35575
f10     1864
dtype: int64

In [14]:
vars_ind_hccv = ['e17', 'e18', 'e19', 'f10']

**Read in the test data**

Saving disk space by reading in strings as factors is great - but how do we get the test data and the smaller train data set to have the same factor levels?

I will read in factors as objects, and then manually convert each object to a factor with the same index for levels and the categories in df_train.

In [15]:
df_train_250k = pd.read_csv(dirRawData + 'train_250k.csv',
                       na_values = 'NA',
                       dtype=dtypes_object)

df_test = pd.read_csv(dirRawData + 'test.csv',
                      na_values = 'NA',
                      dtype=dtypes_object)


In [16]:
# find the categorical vars - this includes the hccv
vars_ind_categorical = list(df_train_1m.columns[df_train_1m.dtypes == 'category'])

# Check how the index of the levels of factors is stored, and print them out
for var in vars_ind_categorical:
    print(df_train_1m[var].cat.categories)
    
# In this version notebook ,the first printed line shows the issue, code C is last
# in the index:
# Index(['A', 'B', 'D', 'F', 'G', 'H', 'C'], dtype='object')

Index(['A', 'B', 'D', 'F', 'G', 'H', 'C'], dtype='object')
Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], dtype='object')
Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'], dtype='object')
Index(['N', 'Y', 'Z'], dtype='object')
Index(['A', 'B', 'C', 'D', 'E'], dtype='object')
Index(['A', 'B', 'C', 'E', 'F'], dtype='object')
Index(['N', 'Y', 'Z'], dtype='object')
Index(['N', 'Y', 'Z'], dtype='object')
Index(['E', 'F', 'D'], dtype='object')
Index(['N', 'Y'], dtype='object')
Index(['N', 'Y'], dtype='object')
Index(['N', 'Y'], dtype='object')
Index(['N', 'Y'], dtype='object')
Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'], dtype='object')
Index(['A', 'B', 'F'], dtype='object')
Index(['Z', 'B'], dtype='object')
Index(['A', 'B', 'C'], dtype='object')
Index(['A', 'B', 'C'], dtype='object')
Index(['A', 'B', 'C'], dtype='object')
Index(['A', 'B', 'C'], dtype='object')
Index(['A', 'B', 'C'], dtype='object')
Index(['A', 'B', 'C'], dtype='object')
Index(['A', 'B', 'C'], dtyp

In [17]:
# Now for each relevant variable in df_test, replace it with a categorical
# Let's also see sizes of datasets before and after

mem_used = df_train_250k.memory_usage().sum() / (2<<29)
print("df_train_250k BEFORE is using {:.3f} gig".format(mem_used))

mem_used = df_test.memory_usage().sum() / (2<<29)
print("df_test BEFORE is using {:.3f} gig".format(mem_used))

for var in vars_ind_categorical:
    var_levels = df_train_1m[var].cat.categories.values
    df_train_250k[var] = pd.Categorical(df_train_250k[var], categories=var_levels, ordered=False)
    df_test[var]       = pd.Categorical(df_test[var],       categories=var_levels, ordered=False)
    
mem_used = df_train_250k.memory_usage().sum() / (2<<29)
print("df_train_250k AFTER is using {:.3f} gig".format(mem_used))

mem_used = df_test.memory_usage().sum() / (2<<29)
print("df_test AFTER is using {:.3f} gig".format(mem_used))


df_train_250k BEFORE is using 0.101 gig
df_test BEFORE is using 0.118 gig
df_train_250k AFTER is using 0.029 gig
df_test AFTER is using 0.031 gig


In [18]:
# Check that the index for a01 is stored in the same order as in df_train.  It is.
df_test['a01'].cat.categories

Index(['A', 'B', 'D', 'F', 'G', 'H', 'C'], dtype='object')

**Define variable lists**

 - var_dep: the target feature
 - vars_notToUse:  the variables that will not be included in the experiments
 - vars_ind: the independent variables
 - vars_ind_numeric: the numeric independent variables
 - vars_ind_hccv: Three high cardinality categorical variables
 - vars_ind_categorical: the categorical independent variables.

In [19]:
vars_all = df_train_1m.columns.values
var_dep = ['target']

vars_notToUse = ['id','unique_id'] 
vars_ind = [var for var in vars_all if var not in (vars_notToUse + var_dep)]

vars_ind_numeric = [var for var in vars_ind if var not in vars_ind_categorical]


In [13]:
srs_missing = df_train_1m.isnull().sum(axis=0)
srs_missing[srs_missing > 0] 
# there are no missing data - but REMEMBER, numeric value -99 is missing!!

Series([], dtype: int64)

**Correlation of the numerical variables**

Most of the numerical variables do not have high correlation between each other. However, there are some set of variables e.g. 'v_num_98' with 'v_num_79' which have high correlation (white squares).

In [ ]:
corr = df_train_1m.loc[1:10000, vars_ind_numeric].corr()
ax = plt.subplots(figsize=(22, 22))
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

In [ ]:
# Deal with missings

In [14]:
dfmiss=df_train_250k[vars_ind_numeric]==-99

In [34]:
dfmiss

,e02,e04,e05,e06,e07,e08,e09,e12,e15,e16,...,f21,f22,f23,f24,f25,f26,f28,f31,f32,unique_id
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
249996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
249997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
249998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [35]:
dfmiss.values.sum()

0

In [15]:
dfmiss[dfmiss==True].count(axis=0) # check how many missings per column

a04    102984
a05     23788
a06     23788
a07     23788
a08      1852
a09      2090
a11     23898
a14     24321
a15     11673
b01     32806
b05     33764
b06    197803
c01      8895
c03      6151
d01     55544
d02     57869
d03     57136
e02         2
e04         0
e05         0
e06         0
e07         0
e08         0
e09         0
e12      1092
e15         0
e16         0
e23         0
f01         0
f02         0
f06         0
f08         0
f11         1
f13         1
f15         0
f16         0
f17         0
f18         0
f19         0
f20         0
f21         0
f22         0
f23         0
f24         0
f25         0
f26         0
f28         0
f31         0
f32         0
dtype: int64

In [20]:
#Remove columns with high number of missings
vars_notToUse = ['id','unique_id','a04','a05','a06','a07','a08','a09','a11','a14','a15','b01','b05','b06','c01','c03','d01','d02','d03']
                 

vars_ind_numeric = [var for var in vars_ind if var not in vars_ind_categorical]

In [ ]:
len(vars)

In [25]:
#Remove missings
df_train_250k = df_train_250k[df_train_250k.e02 != -99]
df_train_250k = df_train_250k[df_train_250k.e12 != -99]
df_train_250k = df_train_250k[df_train_250k.f11 != -99]
df_train_250k = df_train_250k[df_train_250k.f13 != -99]

In [26]:
df_train_250k[vars_ind_numeric].shape

(248905, 49)

In [27]:
df0=df_train_250k[vars_ind_numeric]==0

In [28]:
df0[df0==True].count(axis=0)

a04      3814
a05    218883
a06    212824
a07    221524
a08    243569
a09    232126
a11    222201
a14    123191
a15         0
b01         0
b05       495
b06         4
c01      2531
c03     43063
d01         0
d02         0
d03         0
e02      2956
e04         0
e05      1015
e06         0
e07    243004
e08       155
e09         0
e12      2380
e15      2404
e16         0
e23         0
f01         0
f02         0
f06        11
f08      3613
f11         0
f13         0
f15    219916
f16    232101
f17    238751
f18    247016
f19    245858
f20    232325
f21    247060
f22    237143
f23    248366
f24    247330
f25    247259
f26    247293
f28      3613
f31    230508
f32    232432
dtype: int64

### Save Data

I tried two ways of storing the data.  hd5 file stores and pickle.

We have already been using pickle, a traditional Python way to store any object.  Objects are convered to a stream of bytes and sent to a file.

hd5 filestores:  See https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/

It is efficient for storing homoegeneous blocks of data.  It allows retrieval of specific bits of the data stored in it - unlike pickle where you have to load the whole object even if you only want part of the data.  This means that hd5 is very good for storing huge files.

By the way, our data table is not homogeneous (since it has categories and integers).  Also the categories are not a type it is comfortable with.  So we should not expect good performance. 

The pickle output is smaller (259m vs 295m) and took much less time to store (1.69s vs 7.95s).

For the moment I will continue to use pickle.

In [ ]:
#%%time
#store = pd.HDFStore(dirPData + '01_df_o.h5')
#df_train.to_hdf(store, 'df_train', format="table")
#df_test.to_hdf(store, 'df_test', format="table")
#store.close()

#CPU times: user 5.37 s, sys: 971 ms, total: 6.34 s
#Wall time: 7.95 s

In [29]:
%%time
dict_ = {'df_train': df_train_1m,
         'df_test': df_test}

f_name = dirPData + '01_df_1m.pickle'
with open(f_name, "wb") as f:
    pickle.dump(dict_, f)
del f_name, dict_


CPU times: user 110 ms, sys: 136 ms, total: 246 ms
Wall time: 386 ms


In [30]:
%%time
dict_ = {'df_train': df_train_250k,
         'df_test': df_test}

f_name = dirPData + '01_df_250k.pickle'
with open(f_name, "wb") as f:
    pickle.dump(dict_, f)
del f_name, dict_


CPU times: user 43 ms, sys: 75.9 ms, total: 119 ms
Wall time: 151 ms


In [31]:
# store variable names to pickle
dict_ = {'vars_ind_numeric': vars_ind_numeric,
         'vars_ind_categorical': vars_ind_categorical,
         'vars_ind_hccv': vars_ind_hccv,
         'vars_notToUse': vars_notToUse,
         'var_dep': var_dep}

f_name = dirPData + '01_vars.pickle'
with open(f_name, "wb") as f:
    pickle.dump(dict_, f)
del f_name, dict_

In [29]:
df0

,e02,e04,e05,e06,e08,e09,e12,e15,e16,e23,f01,f02,f06,f08,f11,f13,f28
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
249996,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
249997,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
249998,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
